# Algebraic operators

In [ ]:
# file: mps/register.py
import numpy as np
import mps.state
import mps.mpo
from mps.mpo import MPO
from mps.state import MPS

This notebook implements a number of MPO's that act on an MPS quantum register with 'N' qubits, applying functions of the integers encoded in the register itself. In other words, we create operators of the form
$$\hat{O}_f  = \sum_{s} f(s_1,s_2,\ldots) |s_1,s_2\ldots\rangle\!\langle{s_1,s_2\ldots}|.$$

We have efficient expressions for these operators in the following cases:
* Linear and quadratic functions of the bits, what we call QUBO expressions.
* Polynomials of those linear variables.
* Exponentials of QUBO expressions.

## QUBO operator

Assume that we have a quantum register of $N$ qubits $|s\rangle:=|s_1,s_2\ldots s_N\rangle,$ where $s_i\in\{0,1\}.$ We want to apply a linear-biquadratic function on it $H(J,h)$ given by
$$H = \sum_{ij} J_{ij}s_i s_j + \sum_i h_i s_i.$$

This function constructs the MPO associated to this QUBO operator. To simplify matters, we just realize that $s_i=s_i^2$ and thus the problem may be rewritten as a QUBO with $h=0$ and
$$J_{ij}' = J_{ij} + \delta_{ij}h_i.$$

In [ ]:
# file: mps/register.py

def qubo_mpo(J=None, h=None, **kwdargs):
    """Return the MPO associated to a QUBO operator
         $\sum_i J_{ij} s_i s_j + \sum_i h_i s_i$
    defined by the interaction 'J' and the field 'h'.
    
    Parameters
    ----------
    J        -- Matrix of interactions, or None
    h        -- Magnetic field, or None
    kwdargs  -- Extra arguments for MPO()
    
    Output
    ------
    mpo      -- An object of type MPO
    """
    if J is None:
        #
        # Just magnetic field. A much simpler operator
        if h is None:
            raise Exception('In QUBO_MPO, must provide either J or h')
        #
        data = []
        id2 = np.eye(2)
        for (i,hi) in enumerate(h):
            A = np.zeros((2,2,2,2), dtype=hi.dtype)
            A[0,1,1,1] = hi
            A[1,:,:,1] = id2
            A[0,:,:,0] = id2
            data.append(A)
        A = A[:,:,:,[1]]
        data[-1] = A
        data[0] = data[0][[0],:,:,:]
    else:
        if h is not None:
            J = J + np.diag(h)
        L = len(J)
        id2 = np.eye(2)
        data = []
        for i in range(L):
            A = np.zeros((i+2,2,2,i+3))
            A[0,1,1,1] = J[i,i]
            A[1,:,:,1] = np.eye(2)
            A[0,:,:,0] = np.eye(2)
            A[0,1,1,i+2] = 1.0
            for j in range(i):
                A[j+2,1,1,1] = J[i,j]+J[j,i]
                A[j+2,:,:,j+2] = np.eye(2)
            data.append(A)
        data[-1] = data[-1][:,:,:,[1]]
        data[0] = data[0][[0],:,:,:]
    return MPO(data, **kwdargs)

## Exponential of QUBO

We now provide a slightly different operator, given by the exponential of the QUBO form
$$H = \exp\left[\beta\left( \sum_{ij} J_{ij}s_i s_j + \sum_i h_i s_i\right)\right]$$

This is implemented as an MPOList if $J$ is nonzero, or as an MPO if only $h$ is present. The prefactor $\beta$ may be complex.

In [ ]:
# file: mps/register.py

def qubo_exponential_mpo(J=None, h=None, **kwdargs):
    """Return the MPO associated to the exponential $\exp(\\beta H)$ of 
    the QUBO operator
         $H = \sum_i J_{ij} s_i s_j + \sum_i h_i s_i$
    defined by the interaction 'J' and the field 'h'.
    
    Parameters
    ----------
    J        -- Matrix of interactions, or None
    h        -- Magnetic field, or None
    kwdargs  -- Extra arguments for MPO()
    
    Output
    ------
    mpo      -- An object of type MPO or MPOList
    """
    if J is None:
        #
        # Just magnetic field. A much simpler operator
        if h is None:
            raise Exception('In QUBO_MPO, must provide either J or h')
        #
        data = []
        for (i,hi) in enumerate(h):
            A = np.zeros((1,2,2,1))
            A[0,1,1,1] = np.exp(hi)
            A[0,0,0,0] = 1.0
            data.append(A)
        return MPO(data, **kwdargs)
    else:
        if h is not None:
            J = J + np.diag(h)
        J = (J + J.T)/2
        L = len(J)
        id2 = np.eye(2)
        noop = np.eye(2).reshape(1,2,2,1)
        out = []
        for i in range(L):
            data = [noop] * i
            A = np.zeros((1,2,2,2))
            A[0,1,1,1] = np.exp(β * J[i,i])
            A[0,0,0,0] = 1.0
            for j in range(i+1,L):
                A = np.zeros((2,2,2,2))
                A[1,1,1,1] = np.exp(β * J[i,j])
                A[1,0,0,1] = 1.0
                A[0,0,0,0] = 1.0
                A[0,1,1,0] = 1.0
                data.append(A)
            data[-1] = A[:,:,:,[0]] + A[:,:,:,[1]]
            out.append(MPO(data, **kwdargs))
        return MPOList(out)

## Nonlinear transformations

The following function is a bit out of place, but it also is a nontrivial transformation of the quantum register which works by squaring the wavefunction. This can be done in two ways: as complex
$$\sum_s \psi(s)|s\rangle \to \sum_s \psi(s)^2|s\rangle$$
or with complex conjugate
$$\sum_s \psi(s)|s\rangle \to \sum_s |\psi(s)|^2|s\rangle.$$

More generally, we implement as a product of wavefunctions $\psi(s)\xi(s) \to \psi'(s),$ with or without complex conjugation and with or without simplification.

In [ ]:
# file: mps/register.py

def wavefunction_product(ψ, ξ, conjugate=False, simplify=True, **kwdargs):
    """Implement a nonlinear transformation that multiplies two MPS, to
    create a new MPS with combined bond dimensions. In other words, act
    with the nonlinear transformation <s|ψξ> = ψ(s)ξ(s)|s> or
    <s|ψ*ξ> = ψ*(s)ξ(s)|s>
    
    Arguments
    ---------
    ψ, ξ      -- Two MPS or CanonicalMPS.
    conjugate -- Conjugate ψ or not.
    simplify  -- Simplify the state afterwards or not.
    kwdargs   -- Arguments to simplify() if simplify is True.
    
    Output
    ------
    mps       -- The MPS product ψξ or ψ*ξ.
    """
    
    def combine(A, B):
        # Combine both tensors
        a, d, b = A.shape
        c, d, e = B.shape
        if conjugate:
            A = A.conj()
        D = np.array([np.outer(A[:,i,:].flatten(), B[:,i,:].flatten()) for i in range(d)])
        D = np.einsum('iabce->acibe', np.array(D).reshape(d,a,b,c,e)).reshape(a*c,d,b*e)
        return D

    out = MPS([combine(A,B) for A,B in zip(ψ,ξ)])
    if simplify:
        out = CanonicalMPS(out, center=0, **kwdargs)
        out, _, _ = mps.truncate.simplify(out, **kwdargs)
    return out

## Register transformation

### a) Two's complement

In [ ]:
# file: mps/register.py

def twoscomplement(L, control=0, sites=None, **kwdargs):
    """Return an MPO that performs a two's complement of the selected qubits
    depending on a 'control' qubit in a register with L qubits.
    
    Arguments
    ---------
    L       -- Real size of register
    control -- Which qubit (relative to sites) controls the sign.
               Defaults to the first qubit in 'sites'.
    sites   -- The qubits involved in the MPO. Defaults to range(L).
    kwdargs -- Arguments for MPO.
    
    Returns
    -------
    mpo     -- An MPO object
    """
    
    if sites is not None:
        sites = sorted(sites)
        out = twoscomplement(len(sites), control=sites.index(control), sites=None, **kwdargs)
        return out.extend(L, sites=sites)
    else:
        A0 = np.zeros((2,2,2,2))
        A0[0,0,0,0] = 1.
        A0[1,1,1,1] = 1.
        A = np.zeros((2,2,2,2))
        A[0,0,0,0] = 1.
        A[0,1,1,0] = 1.
        A[1,1,0,1] = 1.
        A[1,0,1,1] = 1.
        data = [A0 if i == control else A for i in range(L)]
        A = data[0]
        data[0] = A[[0],:,:,:] + A[[1],:,:,:]
        A = data[-1]
        data[-1] = A[:,:,:,[0]] + A[:,:,:,[1]]
        return MPO(data, **kwdargs)

## Testing

In [ ]:
# file: mps/test/test_register.py
from mps.register import * 

In [ ]:
# file: mps/test/test_register.py
import unittest
import numpy as np
from mps.test.tools import *
from mps.state import MPS
import scipy.sparse as sp

class TestAlgebraic(unittest.TestCase):
    
    P1 = sp.diags([0.,1.],0)
    i2 = sp.eye(2, dtype=np.float64)
    
    @classmethod
    def projector(self, i, L):
        return sp.kron(sp.eye(2**i), sp.kron(self.P1, sp.eye(2**(L-i-1))))
    
    @classmethod
    def linear_operator(self, h):
        L = len(h)
        return sum(hi * self.projector(i, L) for i,hi in enumerate(h) if hi)
    
    @classmethod
    def quadratic_operator(self, J):
        L = len(J)
        return sum(J[i,j] * (self.projector(i,L) @ self.projector(j,L))
                   for i in range(L) for j in range(L) if J[i,j])
    
    def test_qubo_magnetic_field(self):
        np.random.seed(1022)
        for N in range(1, 10):
            h = np.random.rand(N) - 0.5
            self.assertTrue(similar(qubo_mpo(h=h).tomatrix(),
                                    self.linear_operator(h)))
    
    def test_qubo_quadratic(self):
        np.random.seed(1022)
        for N in range(1, 10):
            J = np.random.rand(N,N) - 0.5
            self.assertTrue(similar(qubo_mpo(J=J).tomatrix(),
                                    self.quadratic_operator(J)))

    def test_product(self):
        np.random.seed(1034)
        for N in range(1, 10):
            ψ = np.random.rand(2**N,2)-0.5
            ψ = ψ[:,0] + 1j*ψ[:,1]
            ψ /= np.linalg.norm(ψ)
            ψmps = MPS.fromvector(ψ,[2]*N)
            ψ = ψmps.tovector()
            
            ξ = np.random.rand(2**N,2)-0.5
            ξ = ξ[:,0] + 1j*ξ[:,1]
            ξ /= np.linalg.norm(ξ)
            ξmps = MPS.fromvector(ξ,[2]*N)
            ξ = ξmps.tovector()
            
            ψξ = wavefunction_product(ψmps, ξmps, simplify=True, normalize=False).tovector()
            self.assertTrue(similar(ψξ, ψ*ξ))
            
            ψcξ = wavefunction_product(ψmps, ξmps, conjugate=True, simplify=False, normalize=False).tovector()
            self.assertTrue(similar(ψcξ, ψ.conj() * ξ))

In [ ]:
suite1 = unittest.TestLoader().loadTestsFromNames(['__main__.TestAlgebraic'])
unittest.TextTestRunner(verbosity=2).run(suite1);

In [ ]:
[1,2,3].index(3)